# Battle Royale Environment Trainer
This notebook is for training Battle Royale agents. MADDPG is used for training the agents.

## Setup Environment Dependencies

In [1]:
import sys
from gym_unity.envs import UnityEnv

%matplotlib inline

print("Python version:")
print(sys.version)
print(sys.executable)

# check Python version
if (sys.version_info[0] < 3):
    raise Exception("ERROR: ML-Agents Toolkit (v0.3 onwards) requires Python 3")

Python version:
3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
/Users/adhipradhana/anaconda3/envs/unity-battle-royale/bin/python


## Start Environment

In [2]:
# Environment name
# Remember to put battle royale environment configuration within the config folder
env_name = "environment/battle-royale"

env = UnityEnv(env_name, worker_id=0, use_visual=False, multiagent=True)

print(str(env))

INFO:mlagents_envs:Connected new brain:
PlayerBrain?team=0
INFO:gym_unity:2 agents within environment.


<UnityEnv instance>


/Users/adhipradhana/anaconda3/envs/unity-battle-royale/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Examine Observation Space

In [3]:
# Examine observation space
observation = env.observation_space
print("Agent observation space type: {}".format(observation))

Agent observation space type: Box(41,)


## Examine Action Space

In [4]:
# Examine action space
action = env.action_space
print("Agent action space type: {}".format(action))

Agent action space type: Box(5,)


## Agents Training
This part shows agent training using MADDPG algoritm

### Setup Algorithm Dependencies

In [5]:
import torch
import visdom
import numpy as np

from utils.MADDPG import MADDPG

### Setup Algoritm Parameters

In [36]:
random_seed = 1234
n_states = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]
n_agents = env.number_agents
n_episode = 10000
max_steps = 100000
buffer_capacity = 1000000
batch_size = 1000
episodes_before_train = 100

### Setup MADDPG

In [37]:
# setup seed
torch.manual_seed(random_seed)
np.random.seed(random_seed)

maddpg = MADDPG(n_agents, n_states, n_actions, batch_size, buffer_capacity, episodes_before_train)

FloatTensor = torch.cuda.FloatTensor if maddpg.use_cuda else torch.FloatTensor

### MADDPG Training

In [ ]:
reward_record = []

for i_episode in range(n_episode):
    # reset environment
    obs = env.reset()
    obs = np.stack(obs)
    
    # convert observation to tensor
    if isinstance(obs, np.ndarray):
        obs = torch.from_numpy(obs).float()
    
    total_reward = 0.0
    rr = np.zeros((n_agents,))
    for i_step in range(max_steps):
        obs = obs.type(FloatTensor)
        actions = maddpg.select_action(obs).data.cpu()
        obs_, reward, done, _ = env.step(actions.tolist())

        reward = torch.FloatTensor(reward).type(FloatTensor)
        obs_ = np.stack(obs_)
        obs_ = torch.from_numpy(obs_).float()
        if i_step != max_steps - 1:
            next_obs = obs_
        else:
            next_obs = None

        total_reward += reward.sum()
        rr += reward.cpu().numpy()
        maddpg.memory.push(obs.data, action, next_obs, reward)
        obs = next_obs

        c_loss, a_loss = maddpg.update_policy()

    maddpg.episode_done += 1
    print('Episode: %d, reward = %f' % (i_episode, total_reward))
    reward_record.append(total_reward)

    if maddpg.episode_done == maddpg.episodes_before_train:
        print('training now begins...')
        print('MADDPG on WaterWorld\n' +
              'scale_reward=%f\n' % scale_reward +
              'agent=%d' % n_agents +
              ', coop=%d' % n_coop +
              ' \nlr=0.001, 0.0001, sensor_range=0.3\n' +
              'food=%f, poison=%f, encounter=%f' % (
                  food_reward,
                  poison_reward,
                  encounter_reward))

## Close Environment

In [7]:
env.close()

INFO:mlagents_envs:Environment shut down with return code 0.


In [14]:
actions = [env.action_space.sample() for agent in range(env.number_agents)]
type(actions)

list